## Vamos trabalhar com os dados abertos do ONS - Operador Nacional do Sistema Elétrico.

### Optei por trabalhar com os dados de Carga de Energia. Definição conforme site: [Dados de carga por subsistema numa data de referência em base diária.](https://dados.ons.org.br/dataset/carga-energia)

### Nessa etapa de transformação, iremos modificar o tipo de dado atribuído às variáveis iniciais. Iremos escolher o mais correto para o tipo de informação transportada.

### Load das bibliotecas e definição do contexto spark.

In [1]:
import os
import pyspark

from pyspark import SparkContext
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import *

In [2]:
sc = SparkContext.getOrCreate()

spark = (SparkSession
      .builder
      .getOrCreate()
     )

print('ApplicationID:', sc.applicationId)

ApplicationID: local-1631037395218


### Caminhos de Leitura e Escrita das Bases de dados

In [3]:
read_path = '../data/refined/01_bind/'

write_path = '../data/refined/02_transform/'

In [4]:
df = spark.read.parquet(read_path)

print(f'Número de registros: {df.count()}')

df.printSchema()

Número de registros: 31649
root
 |-- id_subsistema: string (nullable = true)
 |-- nom_subsistema: string (nullable = true)
 |-- din_instante: string (nullable = true)
 |-- val_cargaenergiamwmed: string (nullable = true)



### Como trabalhamos na etapa anterior com csv, todos os dados foram inicialmente atribuídos como string. Isso será modificado agora de acordo com a informação da variável.

- id_subsistema
- nom_subsistema
- din_instante
- val_cargaeenergia (em MWmed)

#### 'id_subsistema' e 'nom_subsistema' 

In [5]:
df.select('id_subsistema').distinct().orderBy(F.col('id_subsistema')).show()

+-------------+
|id_subsistema|
+-------------+
|            N|
|           NE|
|            S|
|           SE|
+-------------+



### Neste caso, "N", "NE", "S" e "SE" representam uma categoria. É uma boa prática converter esses strings em números. Como são apenas 4 opções, o tipo Byte é suficiente para representar todos os possíveis resultados. A transformação ocorrerá da seguinte forma:

- "N" -> 1
- "NE" -> 2
- "S" -> 3
- "SE" -> 4

In [6]:
df = df.withColumn('id_subsistema', F.when(F.col('id_subsistema') == 'N',1).
                  when(F.col('id_subsistema') == 'NE',2).
                  when(F.col('id_subsistema') == 'S',3).
                  when(F.col('id_subsistema') == 'SE',4).cast(ByteType()))

df.select('id_subsistema').distinct().orderBy(F.col('id_subsistema')).show()

+-------------+
|id_subsistema|
+-------------+
|            1|
|            2|
|            3|
|            4|
+-------------+



### Podemos obter o mesmo resultado utilizando UDF (user defined function)

In [7]:
dic_subsistema = {'N':1, 'NE':2, 'S':3, 'SE':4}

def transform_subsistema(id_subsistema):
    return dic_subsistema[id_subsistema]

udf_transform_subsistema = F.udf(transform_subsistema, ByteType())

In [8]:
# df = df.withColumn('id_subsistema', udf_transform_subsistema(F.col('id_subsistema')))
# df.select('id_subsistema').distinct().orderBy(F.col('id_subsistema')).show()

### 'nom_subsistema" transmite o nome por extenso da categoria 'id_subsistema'. Temos duas alternativas abaixo, optei pela primeira.
- Manter a coluna por uma questão de integridade dos dados.
- Eliminar a colunar por possuir informação redundante.

#### 'din_instante'

### Essa é uma variável de data. Corresponde a data de referência da medição. O tipo mais apropriado é o DateType. É necessário verificar qual o formato da data na string. Suspeita-se que seja aaaa-MM-dd.

In [9]:
df = df.withColumn('len_din_instante', F.length(F.col('din_instante')).cast(IntegerType()))

print('Maiores tamanhos de string de data e exemplos:')
df.select('len_din_instante', 'din_instante').orderBy(F.col('len_din_instante').desc()).show(5)

print('Menores tamanhos de string de data e exemplos:')
df.select('len_din_instante', 'din_instante').orderBy(F.col('len_din_instante')).show(5)

print('Valores únicos de string de data:')
df.select('len_din_instante').dropDuplicates().show()

Maiores tamanhos de string de data e exemplos:
+----------------+-------------------+
|len_din_instante|       din_instante|
+----------------+-------------------+
|              19|2013-01-01 00:00:00|
|              19|2013-01-02 00:00:00|
|              19|2013-01-01 00:00:00|
|              19|2013-01-01 00:00:00|
|              19|2013-01-01 00:00:00|
+----------------+-------------------+
only showing top 5 rows

Menores tamanhos de string de data e exemplos:
+----------------+------------+
|len_din_instante|din_instante|
+----------------+------------+
|              10|  2006-01-01|
|              10|  2006-01-01|
|              10|  2006-01-01|
|              10|  2006-01-01|
|              10|  2006-01-02|
+----------------+------------+
only showing top 5 rows

Valores únicos de string de data:
+----------------+
|len_din_instante|
+----------------+
|              19|
|              10|
+----------------+



#### Neste caso, apenas nos interessa os 10 primeiros caracteres.

In [10]:
df = df.withColumn('din_instante', F.substring(F.col('din_instante'),1,10))
df = df.withColumn('din_instante', F.to_date(F.col('din_instante'), 'yyyy-MM-dd'))
df = df.drop('len_din_instante')

#### 'val_cargaenergiamwmed'

### Neste caso, trata-se de uma variável númerica com parte decimal. O tipo indicado então é o DoubleType.

In [12]:
df = df.withColumn('val_cargaenergiamwmed', F.col('val_cargaenergiamwmed').cast(DoubleType()))

### O printschema abaixo indica que os tipos foram alterados.

In [13]:
print(f'Número de registros: {df.count()}')

df.printSchema()

Número de registros: 31649
root
 |-- id_subsistema: byte (nullable = true)
 |-- nom_subsistema: string (nullable = true)
 |-- din_instante: date (nullable = true)
 |-- val_cargaenergiamwmed: double (nullable = true)



### Escrita da base

In [14]:
df.write.parquet(write_path, mode= 'overwrite')

In [15]:
sc.stop()